# Cleaning US postal address data

22 09 24

---

US postal address data are patchy in terms of coverage and also messy, with large numbers of errors in postcodes, town names and street names. This notebook goes through steps to clean-up (to some degree) the US postal address data from OpenAddresses. Based on the number of addresses in this dataset it seems to be the most comprehensive source of US address data.

## Steps

1) Load data from OpenAddresses and filter to the required state, e.g., 'CA' for California
2) Impute the city name based on the name of the file
3) Append the state name
4) For each point (latitude, longitude value) find the nearest road
5) Create an imputed_road column to fix any incorrect road assignments in the OpenAddresses file

## References

- US Census places shapefile: https://www.census.gov/cgi-bin/geo/shapefiles/index.php
- US postcodes shapefile: https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2020&layergroup=ZIP+Code+Tabulation+Areas
- OpenAddresses: https://openaddresses.io/

## Other data sources
https://www.arcgis.com/home/item.html?id=2202c1cd6708441f987ca5552f2d9659

In [31]:
from os import listdir
from re import match

import numpy as np
import geopandas as gpd
import pandas as pd
import polars as pl

import matplotlib.pyplot as plt

### OpenAddresses Data Cleaning functions

In [32]:
def city_from_filename(filename):
    if 'of' in filename:
        town_name = filename.split('/')[-1].split('of')[1].split('-')[0][1:].replace('_', ' ').upper()
    elif 'statewide' not in filename:
        town_name = filename.split('-add')[0].replace('_', ' ').upper()
    else:
        town_name = ''
    return town_name

def valid_postcode(postcode_string):
    """
    Check to see if postcode is valid 5 digit us postcode
    """
    try:
        postcode_clean = int(float(postcode_string))
        if (postcode_clean == float(postcode_string)):
            if match('^[0-9]{5}$', str(postcode_clean)):
                return True
            else:
                return False
        else:
            return False
    except:
        return False

In [33]:
def wrangle_address_file(filenames, state_name, postcodes_filename, places_file):
    """
    Given a list of files for a given state, extract the relevant columns and join to postcode
    and places files to get city and postcode names

    Args
    ----
    filenames (list of str): list of filenames for only a single US state
    state_name (str): name of the state (abbreviated)
    postcodes_filename (str): path of postcodes file for the state
    places_file (str): path of places file for the state

    Returns
    -------

    df_all (pd.DataFrame): combined address data for the state
    """
    gdfs = []

    print(f"Loading postcode file: {postcodes_filename}")
    gdf_postcodes = gpd.read_file(postcodes_filename)
    
    print(f"Loading cities file: {places_filename}")
    gdf_places = gpd.read_file(places_filename)
    
    for filename in filenames:
        print(f'Loading address data for: {filename}')
        # get the city name from the filename
        town_name = city_from_filename(filename)
        
        gdf = (
            gpd
            .read_file(f'{folder_name}/{filename}')
            .assign(latitude=lambda df_: df_.geometry.y,
                    longitude=lambda df_: df_.geometry.x,
                    state=state_name,
                    city2=town_name)
            .loc[:, ['state', 'number', 'street', 'unit', 'city', 'city2', 'district', 'postcode', 'latitude', 'longitude', 'geometry']]
            .to_crs(gdf_postcodes.crs)
        )
        # join to postcodes file to get postcode
        gdf_new = (
            gpd
            .sjoin(gdf, gdf_postcodes, how='left', predicate='within')
            .loc[:, ['state', 'number', 'street', 'unit', 'city', 'city2', 'district', 'postcode', 'latitude', 'longitude', 'geometry', 'ZCTA5CE20']]
        )
        # join to places file to get city
        gdf_new = (
            gpd
            .sjoin(gdf_new, gdf_places, how='left', predicate='within')
            .loc[:, ['state', 'number', 'street', 'unit', 'city', 'city2', 'district', 'postcode', 'latitude', 'longitude', 'ZCTA5CE20', 'NAME', 'geometry']]
        )
        gdfs.append(gdf_new)
        
    df_all = pd.concat(gdfs)
    return df_all 

In [34]:
# load geojson
state_name = 'ca'
folder_name = f'/Users/alexlee/Desktop/Data/geo/open_addresses/{state_name}'

filenames = [filename for filename in listdir(folder_name) if (filename.endswith('json') and 'address' in filename)]

## Wrangle US state data

In [15]:
postcodes_filename = '/Users/alexlee/Desktop/Data/geo/us/shapefiles/tl_2020_us_zcta520/tl_2020_us_zcta520.shp'
places_folder = '/Users/alexlee/Desktop/Data/geo/us/shapefiles/places/california/'

places_file = [filename for filename in listdir(places_folder) if filename.endswith('.shp')]
places_filename = f'{places_folder}{places_file[0]}' # set the filename of the shapefile for the state

In [1]:
# clean the data
addresses_all = wrangle_address_file(filenames, state_name.upper(), postcodes_filename, places_filename)

In [2]:
addresses_all.query('street == ""').sample(1)

In [18]:
# no city, no postcode
addresses1 = (
    addresses_all
    .query('city.str.strip() == ""').query('postcode.str.strip() == ""')
    .assign(city=lambda df_: df_.city2,
            postcode=lambda df_: df_.ZCTA5CE20)
    .loc[:, ['number', 'street', 'unit', 'city', 'state', 'postcode', 'latitude', 'longitude', 'geometry']]
)

# no city, has postcode
addresses2 = (
    addresses_all
    .query('city.str.strip() == ""').query('postcode.str.strip() != ""')
    .assign(city=lambda df_: df_.city2)
    .loc[:, ['number', 'street', 'unit', 'city', 'state', 'postcode', 'latitude', 'longitude', 'geometry']]
)

# has city, no postcode
addresses3 = (
    addresses_all
    .query('city.str.strip() != ""').query('postcode.str.strip() == ""')
    .assign(postcode=lambda df_: df_.ZCTA5CE20)
    .loc[:, ['number', 'street', 'unit', 'city', 'state', 'postcode', 'latitude', 'longitude', 'geometry']]
)

# has city, has postcode
addresses4 = (
    addresses_all
    .query('city.str.strip() != ""').query('postcode.str.strip() != ""')
    .loc[:, ['number', 'street', 'unit', 'city', 'state', 'postcode', 'latitude', 'longitude', 'geometry']]
)

In [19]:
# put these all together and create the full address string
addresses_all_final = (
    pd
    .concat([addresses1, addresses2, addresses3, addresses4])
    .drop_duplicates()
    .assign(full_address=lambda df_: df_.number + ' ' + df_.street.str.upper() + ' ' + df_.unit + ' ' + df_.city.str.upper() + ' ' + df_.state.str.upper() + ' ' + df_.postcode)
    .query('postcode.isnull() == False')
    .assign(rowid=lambda df_: np.arange(df_.shape[0]))
    .reset_index().iloc[:, 1:]
)

## Fix postcodes

In [20]:
# check which ones have valid postcodes
valid_postcodes = addresses_all_final.postcode.apply(valid_postcode)

In [21]:
# addresses that do not have a valid postcode
# use USPS postcode derived from lat, lng values
good_addresses = addresses_all_final.loc[valid_postcodes, :]
bad_addresses = addresses_all_final.loc[valid_postcodes == False, :]

In [22]:
bad_addresses.shape

(306472, 11)

In [23]:
gdf_postcodes = gpd.read_file(postcodes_filename)

In [24]:
bad_addresses = gpd.sjoin(bad_addresses, gdf_postcodes, how='left', predicate='within')

In [25]:
bad_addresses_cleaned = (
    bad_addresses
    .assign(postcode=lambda df_: df_.ZCTA5CE20)
    .assign(full_address=lambda df_: df_.number + ' ' + df_.street.str.upper() + ' ' + df_.unit + ' ' + df_.city.str.upper() + ' ' + df_.state.str.upper() + ' ' + df_.postcode)
    .loc[:, ['number', 'street', 'unit', 'city', 'state', 'postcode', 'latitude', 'longitude', 'full_address', 'geometry']]
)

In [26]:
# join the good with the bad
addresses_all_final = pd.concat([good_addresses, bad_addresses_cleaned])

In [27]:
addresses_all_final = addresses_all_final.query('postcode.isnull() == False')
addresses_all_final = addresses_all_final.assign(postcode=lambda df_: df_.postcode.astype(float).astype(int))

## Fix bad street names

E.g., '03rd st' instead of '3rd street'

### Read the shapefiles

In [30]:
# list all the roads shapefiles
filenames = [filename for filename in listdir('/Users/alexlee/Desktop/Data/geo/us/shapefiles/all_roads/') if filename.endswith('.shp')]
folder_name = '/Users/alexlee/Desktop/Data/geo/us/shapefiles/all_roads'

gdfs = []

# extract all the roads data
for road_filename in filenames:
    print(f"Reading {road_filename}")
    filename = f'{folder_name}/{road_filename}'
    gdf = gpd.read_file(filename).query('FULLNAME.isnull() == False')
    gdfs.append(gdf)

gdf_roads = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

### Append new road name to file

In [ ]:
# convert the postal address data to a GeoDataFrame
gdf_addresses = (
    gpd
    .GeoDataFrame(
        addresses_all_final
        .assign(geometry=gpd.points_from_xy(x=addresses_all_final.longitude, y=df_ca.latitude, crs=gdf_all.crs)))
)

In [ ]:
# find the nearest road to each of the postal addresses
gdf_nearest = (
    gpd
    .sjoin_nearest(gdf_ca.to_crs('EPSG:3310'), gdf_all.to_crs('EPSG:3310'), how='left', max_distance=None)
    .loc[:, ['rowid', 'number', 'street', 'unit', 'city', 'state', 'postcode',
             'latitude', 'longitude', 'full_address', 'index_right',
             'LINEARID', 'FULLNAME', 'RTTYP', 'MTFCC']]
)

roads_grouped = gdf_nearest.loc[:, ['street', 'postcode', 'FULLNAME']].groupby(['street', 'postcode', 'FULLNAME'], as_index=False).size()

roads_best_match = (
    roads_grouped
    .sort_values(by=['street', 'postcode', 'size'], ascending=False)
    .groupby(['street', 'postcode'], as_index=False)
    .first()
)

# mapping between the road_postcode name and the road name
roads_best_match = roads_best_match.assign(street_postcode=lambda df_: df_.street + '_' + df_.postcode.astype(str))

road_to_best_road = dict(roads_best_match.loc[:, ['street_postcode', 'FULLNAME']].values)

In [ ]:
addresses_all_final = (
    addresses_all_final
    .assign(street_postcode=lambda df_: df_.street + '_' + df_.postcode.astype(str))
    .assign(street2=lambda df_: df_.street_postcode.map(road_to_best_road).str.upper().str.strip())
)

In [18]:
# remove the 'NONE' ones
addresses_all_final = addresses_all_final.query('street != "NONE"')

# remove the 'UNASSIGNED' ones
addresses_all_final = addresses_all_final.query('street.str.contains("UNASSIGNED") == False')

addresses_all_final = (
    addresses_all_final
    .query('street != "0"')
)

In [21]:
# find where the mismatches are between street and street2
mismatches = addresses_all_final.drop_duplicates().query('street != street2')

mismatches = (
    mismatches
    .assign(street=lambda df_: df_.street2)
    .assign(full_address=lambda df_: df_.number + ' ' + df_.street.str.upper() + ' ' + df_.unit + ' ' + df_.city.str.upper() + ' ' + df_.state.str.upper() + ' ' + df_.postcode.astype(str))
)

In [25]:
addresses_all_cleaned = (
    pd.concat([mismatches, addresses_all_final.drop_duplicates()])
    .reset_index()
    .iloc[:, 1:]
)

In [26]:
addresses_all_cleaned = addresses_all_cleaned.assign(rowid=np.arange(addresses_all_cleaned.shape[0]))

In [28]:
addresses_all_cleaned.shape

(25288007, 12)

In [38]:
# add the additional addresses where there is a mismatch between street and street2

## Write the data

In [41]:
# write the results
(
    addresses_all_cleaned
    .loc[:, ['rowid', 'number', 'street', 'unit', 'city', 'state', 'postcode', 'latitude', 'longitude', 'full_address']]
    .to_parquet('/Users/alexlee/Desktop/Data/geo/us/open_addresses_cleaned/ca_cleaned_190924.parquet')
)

## Spatial joins

In [266]:
gdf_places = gpd.read_file(places_filename)

In [3]:
gdf_places.query('NAME == "Woodbourne"').geometry.iloc[0]

## Visualise data

In [347]:
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster

In [ ]:
# Select one polygon (e.g., by index)
#selected_polygon = shapefile.query('NAME == "Langhorne"')
roads_polygon = roads_shapefile.query("FULLNAME == 'Glendale St'")

# Get the centroid of the selected polygon to initialize the map
centroid = roads_polygon.geometry.centroid
map_center = [centroid.y.mean(), centroid.x.mean()]

# Initialize the map centered around the selected polygon
my_map = folium.Map(location=map_center, zoom_start=16)

# Add selected polygon to the map
folium.GeoJson(roads_polygon).add_to(my_map)

# Display the map in the notebook
my_map
